In [9]:
# 전처리 동안 사용할 라이브러리 종합
import pandas as pd
import numpy as np

import re
import copy
import datetime
from tqdm import tqdm

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['font.family'] = 'Malgun Gothic'
sns.set(font="Malgun Gothic", 
        rc={"axes.unicode_minus":False},
        style='darkgrid')

import warnings
warnings.filterwarnings('ignore')

## 전처리

### 공시자료 전처리

In [10]:
company_info = pd.read_csv("../data/raw_data/all_ent.csv", encoding='cp949')
company_info = company_info[company_info["회사명"] == "삼성전자"]
company_info.reset_index(drop=True, inplace=True)

In [11]:
company_info.columns

Index(['법인구분', '종목코드', '발행주식 총수', '보고구분', '보고자 구분', '보고자 명칭', '임원', '직위명',
       '주요주주', '변동사유', '변동일', '특정증권등의 종류', '변동전', '증감', '변동후', '회사명', '보고일자'],
      dtype='object')

In [12]:
use_cols = [
    '보고일자',
    '변동일',
    '회사명',
    '보고자 명칭',
    '종목코드',
    '보고구분',
    '보고자 구분',
    '임원',
    '직위명',
    '주요주주',
    '변동사유',
    '특정증권등의 종류',
    '변동전',
    '증감',
    '변동후',
    '발행주식 총수']
company_info = company_info[use_cols]

In [13]:
company_info.head()

,보고일자,변동일,회사명,보고자 명칭,종목코드,보고구분,보고자 구분,임원,직위명,주요주주,변동사유,특정증권등의 종류,변동전,증감,변동후,발행주식 총수
0,20200204,2020년 01월 21일,삼성전자,정문학,5930,신규,개인(국내),비등기임원,상무,-,신규선임(+),보통주,0,500,500,6792669250
1,20180529,2018년 05월 02일,삼성전자,이명진,5930,변동,개인(외국),비등기임원,부사장,-,주식분할,보통주,270,13230,13500,7322953700
2,20200330,2020년 03월 24일,삼성전자,이치훈,5930,신규,개인(국내),비등기임원,상무,-,장내매수(+),보통주,0,25,25,6792669250
3,20200330,2020년 03월 25일,삼성전자,이치훈,5930,신규,개인(국내),비등기임원,상무,-,장내매수(+),보통주,25,100,125,6792669250
4,20200330,2020년 03월 25일,삼성전자,이치훈,5930,신규,개인(국내),비등기임원,상무,-,장내매수(+),보통주,125,25,150,6792669250


In [14]:
# 종목코드 정리
company_info["종목코드"] = company_info["종목코드"].apply(lambda x: str(x).zfill(6))

# 변동일 양식 정리
company_info["변동일"] = company_info["변동일"].apply(lambda x: re.sub(r"[^0-9]","",x))

# str을 datetime 형식으로 변경
company_info.loc[:, "보고일자":"변동일"] = company_info.loc[:, "보고일자":"변동일"].applymap(lambda x: datetime.datetime.strptime(str(x),"%Y%m%d"))

# 일단은 당장 안쓸꺼므로 제거. 나중에 쓸수도 있음.
company_info.drop(columns="변동일", inplace=True)

# 보고자명칭 가운데 공백 제거
company_info["보고자 명칭"] = company_info["보고자 명칭"].apply(lambda x: x.replace(" ",""))

# 각 공시를 그룹화하기 위해 인덱스 생성
company_info["인덱스"] = company_info[["보고일자","보고자 명칭","보고구분"]].apply(lambda x: '_'.join(x.values.astype(str)), axis=1)
company_info.drop(columns="보고자 명칭", inplace=True)

# 데이터프레임 컬럼순서 변경
col0=company_info.columns[-1:].to_list()
col1=company_info.columns[:-1].to_list()
new_col=col0+col1
company_info = company_info[new_col]
new_col

# 인덱스별로 정렬
company_info.sort_values("인덱스", ascending=False, ignore_index=True, inplace=True)

# 인덱스별로 groupby하면서 sum
temp = company_info[["인덱스","변동전","증감","변동후"]].groupby("인덱스").sum("증감")
temp.reset_index(inplace=True)
temp.sort_values("인덱스", ascending=False, ignore_index=True, inplace=True)

company_info.drop_duplicates(subset="인덱스", ignore_index=True, inplace=True)
company_info.drop(columns=["변동전", "증감", "변동후"], inplace=True)
company_info = pd.merge(company_info, temp, how='left', left_on='인덱스', right_on='인덱스')
company_info.set_index("인덱스", inplace=True)

company_info.head()

,보고일자,회사명,종목코드,보고구분,보고자 구분,임원,직위명,주요주주,변동사유,특정증권등의 종류,발행주식 총수,변동전,증감,변동후
인덱스,,,,,,,,,,,,,,
2022-10-24 00:00:00_김윤수_신규,2022-10-24,삼성전자,005930,신규,개인(국내),비등기임원,부사장,-,신규보고(+),우선주,6792669250,0,100,100
2022-10-24 00:00:00_김윤수_변동,2022-10-24,삼성전자,005930,변동,개인(국내),비등기임원,부사장,-,장내매수(+),우선주,6792669250,100,200,300
2022-10-18 00:00:00_이동하_신규,2022-10-18,삼성전자,005930,신규,개인(국내),비등기임원,상무,-,장내매수(+),보통주,5969782550,0,370,370
2022-10-18 00:00:00_박훈철_변동,2022-10-18,삼성전자,005930,변동,개인(국내),비등기임원,상무,-,장내매수(+),우선주,6792669250,176,211,387
2022-10-17 00:00:00_박태훈_변동,2022-10-17,삼성전자,005930,변동,개인(국내),비등기임원,상무,-,장내매수(+),보통주,6792669250,600,208,808


In [15]:
company_info.info()

<class 'pandas.core.frame.DataFrame'>
Index: 691 entries, 2022-10-24 00:00:00_김윤수_신규 to 2015-01-13 00:00:00_최진원_변동
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   보고일자       691 non-null    datetime64[ns]
 1   회사명        691 non-null    object        
 2   종목코드       691 non-null    object        
 3   보고구분       691 non-null    object        
 4   보고자 구분     691 non-null    object        
 5   임원         691 non-null    object        
 6   직위명        691 non-null    object        
 7   주요주주       691 non-null    object        
 8   변동사유       691 non-null    object        
 9   특정증권등의 종류  691 non-null    object        
 10  발행주식 총수    691 non-null    int64         
 11  변동전        691 non-null    int64         
 12  증감         691 non-null    int64         
 13  변동후        691 non-null    int64         
dtypes: datetime64[ns](1), int64(4), object(9)
memory usage: 81.0+ KB


In [16]:
# 저장.
file_name = "../data/preprocessed_data/samsung.csv"
company_info.to_csv(file_name, encoding='cp949')